# 벽 부수고 이동하기

소요시간: 3시간

작성일시: 24.06.21 20:51:53

출처: https://www.acmicpc.net/problem/2206

### 접근 방법
시작점에서 각 칸으로의 최단거리를 구하고, 반대로 끝점에서 최단거리를 구한다. (bfs 2번 수행)

벽으로 구역이 나뉘어있는지 확인하기 위해 bfs를 수행한다.

모든 벽을 조회하면서, 벽을 제거했을 때 생성되는 새로운 경로가 시작점과 끝점을 이을 수 있는지 확인하고, 가능하다면 시작점에서의 거리와 끝점에서의 거리 + 1(벽)로 최단거리를 갱신해나간다.

### 시간 복잡도 분석
O(n^2)

### 새로 알게 된 것
nodejs에는 내장 queue 자료형이 없다.

bfs를 사용해야 할 땐 queue 대신 stack, new stack 2개로 queue의 역할을 대신하는 것이 좋다.

queue를 직접 구현해도 되지만, 최적화가 되지 않아서 실행시간이 늘어난다.

### 주의할 점
.

### 기타 코멘트
낮은 실행 시간을 기록한 다른 사람의 풀이를 보았다.

bfs 여러번 돌고 벽을 순회하며 최단거리를 계산하지 않아도, 벽을 깼을 때의 최단거리 board와 벽을 딱 하나 깼을 때의 최단거리 board를 두고 bfs를 1번 수행하면 답을 구할 수 있다.


In [ ]:
const fs = require('fs')

const input = fs.readFileSync('input.txt').toString().trim().split('\n')
const [N, M] = input[0].split(' ').map((element) => Number(element))
const board = []
for (let i = 1; i <= N; i++) {
    board.push([...input[i].trim()].map((element) => Number(element)))
}
const group = Array.from({ length: N }, () => Array.from({ length: M }, () => 0))
const fromEnd = Array.from({ length: N }, () => Array.from({ length: M }, () => 1E9))
const fromStart = Array.from({ length: N }, () => Array.from({ length: M }, () => 1E9))
const RDLU = [[0, 1], [1, 0], [0, -1], [-1, 0]]

function isGridIn(x, y) {
    return 0 <= x && x < N && 0 <= y && y < M
}

function checkGroup() {
    function bfs(x, y, groupId) {
        let newQ = [[x, y]]
        let q = []
        visited[x][y] = true
        while (newQ.length) {
            q = newQ
            newQ = []
            while (q.length) {
                let [i, j] = q.pop()
                group[i][j] = groupId
                for (let [dx, dy] of RDLU) {
                    let nx = i + dx
                    let ny = j + dy
                    if (isGridIn(nx, ny) && !visited[nx][ny] && !board[nx][ny]) {
                        visited[nx][ny] = true
                        newQ.push([nx, ny])
                    }
                }
            }
        }
    }
    const visited = Array.from({ length: N }, () => Array.from({ length: M }, () => false))
    let groupId = 1
    for (let i = 0; i < N; i++) {
        for (let j = 0; j < M; j++) {
            if (!visited[i][j] && !board[i][j]) {
                bfs(i, j, groupId)
                groupId++
            }
        }
    }
}

function measureDistance(isFromEnd) {
    const fromWhere = isFromEnd ? fromEnd : fromStart
    const start = isFromEnd ? [N - 1, M - 1] : [0, 0]
    const visited = Array.from({ length: N }, () => Array.from({ length: M }, () => false))
    let q = []
    let newQ = [[...start, 1]]
    visited[start[0]][start[1]] = true
    while (newQ.length) {
        q = newQ
        newQ = []
        while (q.length) {
            const [x, y, c] = q.pop()
            fromWhere[x][y] = c
            for (let [dx, dy] of RDLU) {
                let nx = x + dx
                let ny = y + dy
                if (isGridIn(nx, ny) && !visited[nx][ny] && !board[nx][ny]) {
                    visited[nx][ny] = true
                    newQ.push([nx, ny, c + 1])
                }
            }
        }
    }
}

checkGroup()
measureDistance(true)
measureDistance(false)
let distance = fromEnd[0][0]

for (let i = 0; i < N; i++) {
    for (let j = 0; j < M; j++) {
        if (board[i][j]) {
            for (let k = 0; k < 3; k++) {
                for (let l = k + 1; l < 4; l++) {
                    let [d1x, d1y] = RDLU[k]
                    let [d2x, d2y] = RDLU[l]
                    let [n1x, n1y] = [i + d1x, j + d1y]
                    let [n2x, n2y] = [i + d2x, j + d2y]
                    if (isGridIn(n1x, n1y) && isGridIn(n2x, n2y) && !board[n1x][n1y] && !board[n2x][n2y]) {
                        const startGroupId = group[0][0]
                        const endGroupId = group[N - 1][M - 1]
                        if (group[n1x][n1y] === startGroupId && group[n2x][n2y] === endGroupId) {
                            distance = Math.min(distance, fromStart[n1x][n1y] + fromEnd[n2x][n2y] + 1)
                        }
                        if (group[n1x][n1y] === endGroupId && group[n2x][n2y] === startGroupId) {
                            distance = Math.min(distance, fromEnd[n1x][n1y] + fromStart[n2x][n2y] + 1)
                        }
                    }
                }
            }
        }
    }
}

if (distance === 1E9) {
    distance = -1
}

console.log(distance)